In [1]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import torchmetrics
import pandas as pd

In [9]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
region_names = ["brain"]
studies = ["Affine", 
           "SyN",
           "OP",
          ]

method_labels = []
method_labels.append(sorted(glob(os.path.join('dataset3', 'Femina3', 'Mask', "*_affine.nii.gz"))))
method_labels.append(sorted(glob(os.path.join('dataset3', 'Femina3', 'Mask', "*_affine_deformable.nii.gz"))))
method_labels.append(sorted(glob(os.path.join('output', 'Femina3', 'test', 'Mask', '*.nii.gz'))))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
values = [0,0,0,0,0,0,0,0,0,0,0]

list_dict = []
for i in range(len(studies)):
    method = studies[i]
    print(method, end='\r')
    for j in range(len(method_labels[i])):
        id = method_labels[i][j].split('/')[-1].split('_')[0] + '_' + method_labels[i][j].split('/')[-1].split('_')[1]     
        labels = torch.from_numpy(nib.load(method_labels[i][j]).get_fdata().ravel()).type('torch.LongTensor')
        struct = region_names[0]
        volume = ((labels==1).sum().item())*(0.12**3)
        line_dict = {"method": method, "struct": struct, "mouse": id, "volume": volume}
        list_dict.append(line_dict)
df = pd.DataFrame(list_dict)
df.to_csv("dataset3/Femina3/feminad3_volume.csv")